In [18]:
symbol = 'BANKNIFTY'
lot_size = 25
# today='26-Sep-2022'
day = 26
month ='Sep'
year = '2022'

In [19]:
if day < 10 :
    val ='0'+str(day)
    today = val+"-"+month+"-"+year
else:
    today = str(day)+"-"+month+"-"+year
# today

'26-Sep-2022'

In [20]:
# symbol = 'BANKNIFTY'
# lot_size = 25

def get_today_date():
    from datetime import datetime
    date_str = str(datetime.now())
    today=datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S.%f')
    return today.strftime('%d-%b-%Y')

def fce(df_eod1):
    if df_eod1["CE_OI%"]>0 and df_eod1["CE_LTP%"]>0:
        val= "LB"
    elif df_eod1["CE_OI%"]<0 and df_eod1["CE_LTP%"]<0:
        val="LW"
    elif df_eod1["CE_OI%"]<0 and df_eod1["CE_LTP%"]>0:
        val="SC"
    elif df_eod1["CE_OI%"]>0 and df_eod1["CE_LTP%"]<0:
        val="SB"
    else:
        val=np.nan
    return val
def fpe(df_eod1):
    if df_eod1["PE_OI%"]>0 and df_eod1["PE_LTP%"]>0:
        val= "LB"
    elif df_eod1["PE_OI%"]<0 and df_eod1["PE_LTP%"]<0:
        val="LW"
    elif df_eod1["PE_OI%"]<0 and df_eod1["PE_LTP%"]>0:
        val="SC"
    elif df_eod1["PE_OI%"]>0 and df_eod1["PE_LTP%"]<0:
        val="SB"
    else:
        val=np.nan
    return val

def colour_numbers(series):
    red    = 'background-color: tomato;'
    orange = 'background-color: olive;' 
    blue = 'background-color: royalblue;' 
    green = 'background-color: seagreen;' 

    default = ''

    # note multiple else ..if conditions
    return [red if e == 'SB' else green if e == 'LB' else orange if e == 'LW' else blue if e == 'SC' else default for e in series]  


def red_font_negatives(series):
    highlight = 'color: red;'#,'font-weight: bold;'
    default = 'color: green;'
    return [highlight if e < 0 else default for e in series]

def blue_font(series):
    highlight = 'color: royalblue;'#,'font-weight: bold;'
    default = 'color: green;'
    return [highlight if e > 0 else default for e in series]

def bold_max_value_in_series(series):
    highlight = 'font-weight: bold;'
    default = ''

    return [highlight if e > 0 else default for e in series]

def highlight_col(x):
    #copy df to new - original data are not changed
    df = x.copy()
    #set by condition
    strike=(round(df["UNDERLYING"]/100)*100)
    mask = df['STRIKE'] == strike
    df.loc[mask, :] = 'background-color: yellow'
    df.loc[~mask,:] = 'background-color: ""'
    return df 

def get_path_filename(filename):
    path='D:\Jupyter Python\Stocks-office\json-csv\\'
    newFilename=path+filename
    return newFilename

def get_symbol_df(symbol,lot):
    
    # today = get_today_date()
    filename_ce_option_chain =  symbol +'-CE-OPTION-CHAIN-' + today + ".csv"
    filename_pe_option_chain =  symbol +'-PE-OPTION-CHAIN-' + today + ".csv"
    
    filename_ce_option_chain=get_path_filename(filename_ce_option_chain)
    filename_pe_option_chain=get_path_filename(filename_pe_option_chain)


    df_ce = pd.read_csv(filename_ce_option_chain)
    df_pe = pd.read_csv(filename_pe_option_chain)
    
    df_ce["oi"]=(df_ce["oi"]*lot)
    df_ce["oi_change"]=(df_ce["oi_change"]*lot)
    df_pe["oi"]=(df_pe["oi"]*lot)
    df_pe["oi_change"]=(df_pe["oi_change"]*lot)
    
    return df_ce,df_pe
     
    
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
import yfinance as yf
import numpy as np
import time

######### Pl enter option chain details here ###########
# symbol = 'BANKNIFTY'
# lot_size = 25

# symbol = 'NIFTY'
# lot_size = 50

oc_ce,oc_pe = get_symbol_df(symbol,lot_size)

oc_ce.columns = ['TIME', 'STRIKE', 'EXPIRY', 'Index', 'CE_OI', 'CE_CHNG_IN_OI', 'CE_OI%',
       'CE_IV', 'CE_LTP', 'CE_LTP_CHNG', 'CE_LTP%', 'UNDERLYING']
oc_ce['CE_buildup'] = oc_ce.apply(fce, axis=1)

# oc_ce
oc_pe.columns = ['TIME', 'STRIKE', 'EXPIRY', 'Index', 'PE_OI', 'PE_CHNG_IN_OI', 'PE_OI%',
       'PE_IV', 'PE_LTP', 'PE_LTP_CHNG', 'PE_LTP%', 'UNDERLYING']
oc_pe['PE_buildup'] = oc_pe.apply(fpe, axis=1)

oc_pe1=oc_pe[['STRIKE', 'PE_OI', 'PE_CHNG_IN_OI', 'PE_OI%','PE_IV', 'PE_LTP', 'PE_LTP_CHNG', 'PE_LTP%','PE_buildup']]
# oc_pe1
oc_cepe =pd.merge(oc_ce,oc_pe1,on='STRIKE')
# oc_cepe.head()
oc_cepe_rearrange=oc_cepe[['TIME', 'EXPIRY','CE_buildup', 'CE_OI', 'CE_CHNG_IN_OI', 'CE_OI%','CE_IV', 'CE_LTP', 
                           'CE_LTP_CHNG', 'CE_LTP%',  'STRIKE', 'PE_LTP%','PE_LTP_CHNG','PE_LTP' , 'PE_IV','PE_OI%',
                           'PE_CHNG_IN_OI','PE_OI','PE_buildup','UNDERLYING']]
oc_cepe_rearrange_OI =oc_cepe_rearrange.where(oc_cepe_rearrange["CE_OI"]>5000)
oc_cepe_rearrange_OI=oc_cepe_rearrange_OI.dropna()
oc_cepe_rearrange_OI = round(oc_cepe_rearrange_OI,1)
oc_cepe_rearrange_OI["CE_OI"]=oc_cepe_rearrange_OI["CE_OI"]/100000
oc_cepe_rearrange_OI["CE_CHNG_IN_OI"]=oc_cepe_rearrange_OI["CE_CHNG_IN_OI"]/100000
oc_cepe_rearrange_OI["PE_OI"]=oc_cepe_rearrange_OI["PE_OI"]/100000
oc_cepe_rearrange_OI["PE_CHNG_IN_OI"]=oc_cepe_rearrange_OI["PE_CHNG_IN_OI"]/100000


(oc_cepe_rearrange_OI.style.apply(highlight_col, axis=None)
.format(precision=1) # for 1 point decimals
.apply(red_font_negatives, axis=0, subset=['CE_LTP_CHNG','PE_LTP_CHNG'])
# .apply(red_font_negatives, axis=0, subset=['PE_LTP_CHNG'])
.apply(blue_font, axis=0, subset=['CE_LTP','PE_LTP'])
# .apply(blue_font, axis=0, subset=['PE_LTP'])
.apply(bold_max_value_in_series, axis=0, subset=['STRIKE'])
.apply(colour_numbers, axis=0, subset=['CE_buildup','PE_buildup'])
.highlight_max(axis=0, props='background-color:tomato;', subset=['CE_OI','PE_OI'])
.highlight_max(axis=0, props='background-color:mediumturquoise;', subset=['CE_CHNG_IN_OI','PE_CHNG_IN_OI']))

,TIME,EXPIRY,CE_buildup,CE_OI,CE_CHNG_IN_OI,CE_OI%,CE_IV,CE_LTP,CE_LTP_CHNG,CE_LTP%,STRIKE,PE_LTP%,PE_LTP_CHNG,PE_LTP,PE_IV,PE_OI%,PE_CHNG_IN_OI,PE_OI,PE_buildup,UNDERLYING
5,26-Sep-2022 15:30:00,29-Sep-2022,LW,0.1,-0.0,-10.5,154.5,8747.4,-897.5,-9.3,30000.0,24.5,0.6,3.0,103.4,73.4,2.0,4.7,LB,38666.0
34,26-Sep-2022 15:30:00,29-Sep-2022,LW,0.4,-0.0,-6.2,81.3,4728.0,-911.2,-16.2,34000.0,-12.9,-0.5,3.7,57.1,33.7,0.7,2.9,SB,38666.0
39,26-Sep-2022 15:30:00,29-Sep-2022,LW,0.1,-0.0,-8.0,78.4,4244.1,-851.4,-16.7,34500.0,-3.2,-0.2,4.5,51.7,70.0,0.9,2.2,SB,38666.0
44,26-Sep-2022 15:30:00,29-Sep-2022,LW,0.5,-0.0,-6.5,0.0,3657.8,-987.5,-21.3,35000.0,-17.8,-1.1,4.8,46.7,51.7,3.8,11.1,SB,38666.0
54,26-Sep-2022 15:30:00,29-Sep-2022,LW,0.3,-0.1,-17.0,46.8,2715.9,-917.8,-25.3,36000.0,21.3,2.2,12.6,40.4,49.0,4.5,13.7,LB,38666.0
59,26-Sep-2022 15:30:00,29-Sep-2022,SB,0.2,0.0,8.5,40.4,2232.2,-923.4,-29.3,36500.0,42.4,6.2,21.0,36.9,84.8,6.3,13.7,LB,38666.0
64,26-Sep-2022 15:30:00,29-Sep-2022,LW,0.5,-0.0,-0.4,39.2,1743.2,-907.4,-34.2,37000.0,61.7,15.3,40.0,33.9,60.7,7.7,20.3,LB,38666.0
67,26-Sep-2022 15:30:00,29-Sep-2022,SB,0.1,0.0,10.8,33.6,1452.6,-2032.4,-58.3,37300.0,81.8,27.0,60.0,32.6,4.5,0.1,2.0,LB,38666.0
68,26-Sep-2022 15:30:00,29-Sep-2022,LW,0.1,-0.1,-64.3,33.0,1373.4,-1056.5,-43.5,37400.0,76.1,29.0,67.0,32.3,34.5,0.5,2.1,LB,38666.0
69,26-Sep-2022 15:30:00,29-Sep-2022,SB,0.4,0.1,23.7,34.8,1290.8,-903.5,-41.2,37500.0,88.7,38.0,81.0,31.9,-1.6,-0.2,11.2,SC,38666.0
